In [2]:
library(rhdf5)
library(edgeR)
library(MAST)
library(limma)

Loading required package: limma
Loading required package: SummarizedExperiment
Loading required package: GenomicRanges
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following object is masked from ‘package:limma’:

    plotMA

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, cbind, colMeans, colnames,
    colSums, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, lengths, Map, mapply, match,
    mget, order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, 

In [7]:
input_hdf5 = '../hdf5_data/zheng17_melanomaS2.h5'
h5f = H5Fopen(input_hdf5)

In [8]:
matrix = h5f$matrix
gene_attrs = h5f$gene_attrs
gene_names = h5f$gene_attrs$gene_names
gene_ids = h5f$gene_attrs$gene_ids
cell_groups = h5f$cell_attrs$cell_groups

In [9]:
cell_groups

NULL

In [10]:
h5f

HDF5 FILE
        name /
    filename 

        name       otype dclass         dim
0 cell_attrs H5I_GROUP                     
1 gene_attrs H5I_GROUP                     
2 matrix     H5I_DATASET  FLOAT 1000 x 2216

In [8]:
cell_groups = read.csv( '../analysis_output/melanomaS2/phenograph/clusters.csv', header=TRUE, stringsAsFactors=FALSE)

In [9]:
cell_barcodes = cell_groups$cell_barcode

In [7]:
cell_clusters = cell_groups$cluster

In [8]:
#write.csv(cell_groups, 'r_write_csv_test.csv')

In [9]:
names(h5f$gene_attrs)

[1] "gene_ids"   "gene_names"

In [10]:
names(h5f$cell_attrs)

[1] "cell_names"    "cells_on_rows"

In [11]:
H5Fclose(h5f)

In [12]:
cell_group = 'Group2'

## EdgeR
Binarize the groups into two: the given cell group or not

In [13]:
cgs = sort(unique(cell_clusters))

In [20]:
cgs

[1] 0 1 2 3 4 5 6 7

In [18]:
#deg_res = vector('list')
for (cg in cgs)
{
    res = apply_edger(cg, matrix, cell_clusters)
    lrtFiltered = res
    decided_df = data.frame(decideTestsDGE(lrtFiltered))
    colnames(decided_df) <- "decideTestDGE"
    res = cbind(res$table, res$unshrunk.coefficients) 
    res$Pajd = p.adjust(res$PValue, method = "BH")
    res$cluster_id = cg
    res$gene_names = gene_names
    res$gene_ids = gene_ids
    res = cbind(decided_df, res)
    
    res[c(11, 10, 9, 7, 8,5,4,3,2,1,6)]
    f_name = paste(c('./de_res/DE_cluster', cg,'.csv'), collapse='')
    write.csv(res, file=f_name, row.names=FALSE, quote=FALSE)
}

In [16]:
?write.csv

In [17]:
apply_edger = function (cell_group, matrix, cell_groups)
{
    binary_groups = (cell_groups==cell_group)
    counts = matrix    
    dge <- DGEList(counts = counts)
    group_edgeR <- factor(binary_groups)
    design <- model.matrix(~ group_edgeR)
    dge <- estimateCommonDisp(dge, design = design, trend.method = "none")
    dge <- estimateTrendedDisp(dge, design = design, trend.method = "none")
    dge <- estimateTagwiseDisp(dge, design = design, trend.method = "none")
    fit <- glmFit(dge, design)
    res <- glmLRT(fit)
    
    return (res)
    #pVals <- res$table[,4]
    #names(pVals) <- rownames(res$table)

    #pVals <- p.adjust(pVals, method = "fdr")
    
    #return (pVals)
}

In [58]:
edger_res = apply_edger(4, matrix, cell_clusters)

In [60]:
lrtFiltered = edger_res

In [63]:
summary(decideTestsDGE(lrtFiltered))

   group_edgeRTRUE
-1             309
0              406
1              285

In [119]:
?decideTestsDGE

In [34]:
names(edger_res)

[1] "coefficients"          "fitted.values"         "deviance"             
 [4] "method"                "unshrunk.coefficients" "df.residual"          
 [7] "design"                "offset"                "dispersion"           
[10] "prior.count"           "samples"               "prior.df"             
[13] "AveLogCPM"             "table"                 "comparison"           
[16] "df.test"

In [70]:
temp = edger_res$unshrunk.coefficients

In [71]:
temp2 = cbind(temp, edger_res$table) 

In [72]:
edger_padj = p.adjust(edger_res$table$PValue, method = "BH")

In [73]:
edger_res$table$Padj = edger_padj

In [74]:
edger_res$table$cluster_id = 4

In [75]:
edger_res$table$gene_names = gene_names
edger_res$table$gene_ids = gene_ids


In [76]:
edger_res = cbind(temp, edger_res$table) 

In [80]:
decide_res = decideTestsDGE(lrtFiltered)

In [135]:
ss = show(decide_res)

TestResults matrix
  group_edgeRTRUE
1               1
2               1
3              -1
4              -1
5               1
995 more rows ...


In [137]:
data.frame()

<0 x 0 matrix>

In [143]:
decided_df = data.frame(decideTestsDGE(lrtFiltered))

In [148]:
colnames(decided_df) <- "decideTestDGE"

In [77]:
edger_res = cbind(decided_df, edger_res)

In [ ]:
edger_res$

In [78]:
edger_res

group_edgeRTRUE,(Intercept),group_edgeRTRUE,logFC,logCPM,LR,PValue,Padj,cluster_id,gene_names,gene_ids
1,-9.730043,2.64209285,2.85654993,10.16941,263.03307785,3.744485e-59,9.601243e-58,4,HES4,ENSG00000188290
1,-6.247954,0.66538966,0.93705923,11.59868,21.20968074,4.116799e-06,1.266707e-05,4,ISG15,ENSG00000187608
-1,-8.949423,-1.34331443,-0.97997586,10.17571,6.85303385,8.849088e-03,1.718890e-02,4,TNFRSF18,ENSG00000186891
-1,-8.830293,-1.55430634,-1.14102627,10.19423,10.75045953,1.042534e-03,2.413274e-03,4,TNFRSF4,ENSG00000186827
1,-10.300244,2.46994823,2.23204560,10.09674,125.72829493,3.526006e-29,2.776383e-28,4,MXRA8,ENSG00000162576
0,-10.018934,-0.04565752,-0.02647942,10.07490,0.01630275,8.984005e-01,9.176716e-01,4,VWA1,ENSG00000179403
0,-10.699772,-0.78748853,-0.21883231,10.04418,0.78255421,3.763611e-01,4.518141e-01,4,TNFRSF9,ENSG00000049249
1,-11.866028,2.88678686,1.40980127,10.03640,85.52154453,2.291908e-20,1.441452e-19,4,ERRFI1,ENSG00000116285
0,-9.285289,0.54608994,0.51380211,10.14451,2.15084703,1.424913e-01,2.026903e-01,4,RBP7,ENSG00000162444
-1,-9.482969,-1.09277935,-0.64195881,10.11173,8.41840986,3.714405e-03,7.819800e-03,4,TNFRSF1B,ENSG00000028137


In [54]:
edger_res[c(10, 9, 7, 8,5,4,3,2,1,6)]

group_edgeRTRUE,(Intercept),(Intercept).1,group_edgeRTRUE.1,(Intercept).2,group_edgeRTRUE.2,(Intercept).3,group_edgeRTRUE.3,(Intercept).4,group_edgeRTRUE.4
-5.197126e-01,-8.996463,-8.970493,-1.282342e+00,-9.730043,2.124994e-01,-9.065826,6.633558e-01,-9.090869,2.64209285
-2.488351e+00,-6.057803,-6.281218,7.086138e-01,-6.247954,1.114252e-01,-6.188226,5.088286e-01,-6.209616,0.66538966
-1.842164e+00,-8.895230,-10.258111,3.220969e+00,-8.949423,-2.224111e+00,-8.938093,-2.251196e+00,-8.964319,-1.34331443
-2.133751e+00,-8.774429,-10.035309,3.073403e+00,-8.830293,-1.721705e+00,-8.828486,-2.282227e+00,-8.848979,-1.55430634
-4.950535e+00,-9.574031,-9.602880,-2.972478e+00,-10.300244,-9.999999e+07,-9.629663,2.848380e+00,-10.265264,2.46994823
1.065533e-01,-10.036240,-9.919523,-2.828437e+00,-10.018934,-3.024576e+00,-9.949269,-4.424392e-01,-10.005689,-0.04565752
-1.335087e+00,-10.644826,-11.609436,2.661390e+00,-10.699772,1.112194e+00,-10.882131,-1.241896e+00,-10.708174,-0.78748853
-2.314036e-01,-11.020685,-10.998597,-6.048425e-01,-11.866028,-2.075352e+00,-10.979940,1.168685e+00,-11.144047,2.88678686
-8.198062e-02,-9.222410,-9.126637,-3.383865e+00,-9.285289,-1.948232e+00,-9.166735,-9.999999e+07,-9.185176,0.54608994
-2.076457e+00,-9.418266,-10.031872,1.976274e+00,-9.482969,1.509380e+00,-9.765788,-6.419696e-01,-9.511998,-1.09277935


In [62]:
edger_res$table



[1] "list"

In [22]:
edger_pVals = apply_edger(cell_group, matrix, cell_groups)

In [25]:
edger_pVals

1             2             3             4             5 
 1.355702e-01  3.288851e-03  2.974530e-01  1.986543e-03  3.322146e-01 
            6             7             8             9            10 
 2.969003e-01  4.823096e-01  1.816210e-01  1.571459e-01  2.676890e-01 
           11            12            13            14            15 
 1.553670e-02  7.380602e-01  7.742078e-01  7.224434e-01  3.065065e-01 
           16            17            18            19            20 
 2.415096e-01  8.577756e-01  4.834364e-01  3.009404e-03  1.690185e-01 
           21            22            23            24            25 
 4.638421e-01  3.646450e-01  9.350458e-01  2.633153e-01  6.773022e-01 
           26            27            28            29            30 
 9.563670e-01  7.698422e-01  9.368314e-01  4.873470e-01  1.056007e-02 
           31            32            33            34            35 
 1.168003e-01  6.563542e-01  5.777974e-01  2.721284e-01  6.355605e-01 
           36            37            38            39            40 
 5.244817e-01  4.650196e-01  6.606900e-01  9.788755e-01  5.022017e-01 
           41            42            43            44            45 
 8.386543e-01  3.997320e-01  6.866854e-01  3.291341e-01  4.157543e-02 
           46            47            48            49            50 
 4.893757e-04  7.218198e-01  8.025114e-01  9.744279e-01  6.650132e-01 
           51            52            53            54            55 
 8.897114e-01  7.775446e-01  9.589595e-01  2.972473e-01  8.317356e-01 
           56            57            58            59            60 
 7.167292e-02  5.330282e-01  2.550142e-05  8.115681e-01  2.324230e-01 
           61            62            63            64            65 
 8.193781e-01  8.575543e-01  4.834364e-01  9.626139e-01  2.795033e-01 
           66            67            68            69            70 
 5.587996e-01  4.274683e-01  4.344089e-02  1.869395e-01  5.580341e-01 
           71            72            73            74            75 
 5.722402e-01  9.765808e-01  4.917009e-03  5.587996e-01  4.834364e-01 
           76            77            78            79            80 
 3.185613e-01  9.137721e-01  1.545563e-01  2.410840e-07  1.150186e-01 
           81            82            83            84            85 
 9.113402e-01  7.818400e-07  6.861260e-01  3.254055e-01  2.633153e-01 
           86            87            88            89            90 
 8.968790e-01  1.571459e-01  3.338991e-01  3.622600e-01  2.874634e-02 
           91            92            93            94            95 
 2.804482e-01  4.453195e-01  4.559500e-38  6.196302e-01  6.565941e-01 
           96            97            98            99           100 
 3.111184e-01  2.633153e-01  1.928114e-01  8.892566e-01  8.803621e-01 
          101           102           103           104           105 
 3.057786e-01  1.908755e-01  2.460959e-01  9.368314e-01  4.009728e-01 
          106           107           108           109           110 
 6.355605e-01  3.966625e-01  6.984432e-01  4.871408e-01  3.146053e-01 
          111           112           113           114           115 
 6.120160e-01  4.779519e-01  6.272432e-01  4.712570e-01  9.007553e-01 
          116           117           118           119           120 
 7.224434e-01  9.322604e-01  1.704343e-03  3.111184e-01  7.093558e-01 
          121           122           123           124           125 
 1.433654e-04  3.338991e-01  6.545254e-01  8.317356e-01  1.160439e-01 
          126           127           128           129           130 
 4.823096e-01  4.638421e-01  7.380602e-01  2.721284e-01  2.930578e-03 
          131           132           133           134           135 
 1.921644e-10  1.004240e-04  6.256681e-01  3.974954e-01  1.807166e-01 
          136           137           138           139           140 
 1.437808e-01  4.906198e-01  8.434919e-01  4.307154e-01  2.504082e-01 
          141           142         

In [30]:
evaluate(edger_pVals, cell_group, gene_attrs)

[1] 0.7007392

In [8]:
#EdgeR
binary_groups = (h5f$cell_attrs$cell_groups==cell_group)
counts = matrix 

In [9]:
dge <- DGEList(counts = counts)

In [10]:
group_edgeR <- factor(binary_groups)
design <- model.matrix(~ group_edgeR)
dge <- estimateCommonDisp(dge, design = design, trend.method = "none")
dge <- estimateTrendedDisp(dge, design = design, trend.method = "none")
dge <- estimateTagwiseDisp(dge, design = design, trend.method = "none")


In [11]:
fit <- glmFit(dge, design)
res <- glmLRT(fit)
pVals <- res$table[,4]
names(pVals) <- rownames(res$table)

pVals <- p.adjust(pVals, method = "fdr")

In [16]:
?DGEList

## MAST

In [38]:
counts = matrix
binary_groups = (h5f$cell_attrs$cell_groups==cell_group)

In [47]:
log_counts <- log(counts + 1) / log(2)
fData <- data.frame(names = gene_names)
rownames(fData) <- gene_names;
cData <- data.frame(cond = binary_groups)
rownames(cData) <- colnames(log_counts)

obj <- FromMatrix(as.matrix(log_counts), cData, fData)
colData(obj)$cngeneson <- scale(colSums(assay(obj) > 0))
cond <- factor(colData(obj)$cond)

# Model expression as function of condition & number of detected genes
zlmCond <- zlm(~ cond + cngeneson, obj) 

`fData` has no primerid.  I'll make something up.
`cData` has no wellKey.  I'll make something up.
No dimnames in `exprsArray`, assuming `fData` and `cData` are sorted according to `exprsArray`
..................................................
Done!


In [50]:
summaryCond <- summary(zlmCond)
summaryDt <- summaryCond$datatable

summaryDt <- as.data.frame(summaryDt)
pVals <- unlist(summaryDt[summaryDt$component == "H",4]) # H = hurdle model


Combining coefficients and standard errors
Calculating log-fold changes


In [ ]:
names(pVals) <- unlist(summaryDt[summaryDt$component == "H",1])
pVals <- p.adjust(pVals, method = "fdr")


In [53]:
summaryDt$component

[1] "C"     "C"     "C"     "C"     "C"     "C"     "C"     "C"     "C"    
   [10] "C"     "C"     "C"     "C"     "C"     "C"     "C"     "C"     "C"    
   [19] "C"     "C"     "C"     "C"     "C"     "C"     "C"     "C"     "C"    
   [28] "C"     "C"     "C"     "C"     "C"     "C"     "C"     "C"     "C"    
   [37] "C"     "C"     "C"     "C"     "C"     "C"     "C"     "C"     "C"    
   [46] "C"     "C"     "C"     "C"     "C"     "C"     "C"     "C"     "C"    
   [55] "C"     "C"     "C"     "C"     "C"     "C"     "C"     "C"     "C"    
   [64] "C"     "C"     "C"     "C"     "C"     "C"     "C"     "C"     "C"    
   [73] "C"     "C"     "C"     "C"     "C"     "C"     "C"     "C"     "C"    
   [82] "C"     "C"     "C"     "C"     "C"     "C"     "C"     "C"     "C"    
   [91] "C"     "C"     "C"     "C"     "C"     "C"     "C"     "C"     "C"    
  [100] "C"     "C"     "C"     "C"     "C"     "C"     "C"     "C"     "C"    
  [109] "C"     "C"     "C"     "C"     "C"     "C"     "C"     "C"     "C"    
  [118] "C"     "C"     "C"     "C"     "C"     "C"     "C"     "C"     "C"    
  [127] "C"     "C"     "C"     "C"     "C"     "C"     "C"     "C"     "C"    
  [136] "C"     "C"     "S"     "S"     "S"     "S"     "S"     "S"     "S"    
  [145] "S"     "S"     "S"     "S"     "S"     "S"     "S"     "S"     "S"    
  [154] "S"     "S"     "S"     "S"     "S"     "S"     "S"     "S"     "S"    
  [163] "S"     "S"     "S"     "S"     "S"     "S"     "S"     "S"     "S"    
  [172] "S"     "S"     "S"     "S"     "S"     "S"     "S"     "S"     "S"    
  [181] "S"     "S"     "S"     "S"     "S"     "S"     "S"     "S"     "S"    
  [190] "S"     "S"     "S"     "S"     "S"     "S"     "S"     "S"     "S"    
  [199] "S"     "S"     "S"     "S"     "S"     "S"     "S"     "S"     "S"    
  [208] "S"     "S"     "S"     "S"     "S"     "S"     "S"     "S"     "S"    
  [217] "S"     "S"     "S"     "S"     "S"     "S"     "S"     "S"     "S"    
  [226] "S"     "S"     "S"     "S"     "S"     "S"     "S"     "S"     "S"    
  [235] "S"     "S"     "S"     "S"     "S"     "S"     "S"     "S"     "S"    
  [244] "S"     "S"     "S"     "S"     "S"     "S"     "S"     "S"     "S"    
  [253] "S"     "S"     "S"     "S"     "S"     "S"     "S"     "S"     "S"    
  [262] "S"     "S"     "S"     "S"     "S"     "S"     "S"     "S"     "S"    
  [271] "S"     "S"     "S"     "S"     "D"     "D"     "D"     "D"     "D"    
  [280] "D"     "D"     "D"     "D"     "D"     "D"     "D"     "D"     "D"    
  [289] "D"     "D"     "D"     "D"     "D"     "D"     "D"     "D"     "D"    
  [298] "D"     "D"     "D"     "D"     "D"     "D"     "D"     "D"     "D"    
  [307] "D"     "D"     "D"     "D"     "D"     "D"     "D"     "D"     "D"    
  [316] "D"     "D"     "D"     "D"     "D"     "D"     "D"     "D"     "D"    
  [325] "D"     "D"     "D"     "D"     "D"     "D"     "D"     "D"     "D"    
  [334] "D"     "D"     "D"     "D"     "D"     "D"     "D"     "D"     "D"    
  [343] "D"     "D"     "D"     "D"     "D"     "D"     "D"     "D"     "D"    
  [352] "D"     "D"     "D"     "D"     "D"     "D"     "D"     "D"     "D"    
  [361] "D"     "D"     "D"     "D"     "D"     "D"     "D"     "D"     "D"    
  [370] "D"     "D"     "D"     "D"     "D"     "D"     "D"     "D"     "D"    
  [379] "D"     "D"     "D"     "D"     "D"     "D"     "D"     "D"     "D"    
  [388] "D"     "D"     "D"     "D"     "D"     "D"     "D"     "D"     "D"    
  [397] "D"     "D"     "D"     "D"     "D"     "D"     "D"     "D"     "D"    
  [406] "D"     "D"     "D"     "D"     "D"     "D"     "D"     "D"     "D"    
  [415] "D"     "D"     "D"     "D"     "D"     "D"     "D"     "D"     "D"    
  [424] "D"     "D"     "D"     "D"     "D"     "D"     "D"     "D"     "D"    
  [433] "D"     "D"     "D"     "D"     "D"     "D"     "D"     "D"     "D"    
  [442] "D"     "D"     "D"     "D"     "D"     "D"     "D"     "D"     "D"    
  [4

In [24]:
pVals

named numeric(0)

In [59]:
evaluate(pVals, cell_group)

ERROR: Error in ROCR::prediction(pVals, truth): Number of classes is not equal to 2.
ROCR currently supports only evaluation of binary classification tasks.


## Limma

In [31]:
# *** Running Limma:

Run_Limma = function(rawData, gr1, gr2){
  
    require(limma)

    filtered = apply(rawData,1, function(x) {if(all(x == 0)) return (FALSE) else return(TRUE)})
    FilteredData = rawData[filtered,]

    # Samples' conditions:
    mType <- factor (c(rep("GR1",dim(gr1)[2]) ,rep("GR2",dim(gr2)[2])))

    # Normalization factors from edgeR TMM method:
    nf <- calcNormFactors(FilteredData)
    design <- model.matrix(~mType)

    # Voom transformation for RNA-seq data:
    y <- voom (FilteredData, design, lib.size = colSums(FilteredData)*nf)

    # Linear modeling:
    fit <- lmFit(y,design)
    fit <- eBayes(fit)

    # Summary of the results:
    Limma_results <- topTable(fit,coef=2,n=nrow(fit))
    pVals = Limma_results$P.Value
    pVals <- p.adjust(pVals, method = "fdr")                            
    return (pVals)
  
}

In [32]:
binary_groups = (h5f$cell_attrs$cell_groups==cell_group)

In [33]:
gr1 = matrix[,binary_groups]

In [34]:
gr2 = matrix[,!binary_groups]

In [35]:
dim(gr2)

[1] 1000 2213

In [36]:
limma_res = Run_Limma(matrix, gr1, gr2)

In [37]:
evaluate(pVals = limma_res, cell_group, gene_attrs)

[1] 0.4830124